In [1]:
import os
os.environ["GOOGLE_API_KEY"] = "removed"
os.environ["WEATHER_API_KEY"] = "removed"  
from typing import Annotated
import re
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages




In [2]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [3]:
import os
from langchain.chat_models import init_chat_model



llm = init_chat_model("google_genai:gemini-2.0-flash")
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

In [4]:
graph_builder.add_edge(START, "chatbot")

In [5]:
import requests
import os

def weather_node(state: State):
    last_msg = state["messages"][-1]
    content = last_msg.content if hasattr(last_msg, "content") else last_msg["content"]
    
    
    match = re.search(r"in ([a-zA-Z\s]+)", content)
    city = match.group(1).strip() if match else "London"  
    
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={GOOGLE_API_KEY}&units=metric"
        response = requests.get(url)
        data = response.json()

        if data.get("cod") != 200:
            raise Exception(data.get("message", "Unknown error"))

        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        result = f"The weather in {city} is {weather} with a temperature of {temp}°C."

    except Exception as e:
        result = f"Failed to fetch weather: {str(e)}"

    return {"messages": [{"role": "assistant", "content": result}]}


In [6]:
import re

def calculator(state: State):
    last_msg = state["messages"][-1]["content"]
    try:
        result = eval(last_msg, {"__builtins__": {}}, {})
        response = f"Result: {result}"
    except Exception as e:
        response = f"Calculator error: {e}"
    return {"messages": [{"role": "assistant", "content": response}]}


In [7]:
def router(state: State):
    last_msg = state["messages"][-1]
    content = last_msg.content if hasattr(last_msg, "content") else last_msg["content"]
    
    if re.search(r"weather|temperature|forecast", content, re.IGNORECASE):
        return {"next": "weather"}
    elif re.match(r"^[\d\s\.\+\-\*\/\(\)]+$", content):
        return {"next": "calculator"}
    else:
        return {"next": "chatbot"}


In [8]:
graph_builder.add_node("calculator", calculator)
graph_builder.add_node("router", router)

graph_builder.set_entry_point("router")

graph_builder.add_node("weather", weather_node)
graph_builder.add_conditional_edges("router", router)

In [11]:
app = graph_builder.compile()

# Example usage
output = app.invoke({"messages": [{"role": "user", "content": "weather in bengaluru"}]})
print(output)
output = app.invoke({"messages": [{"role": "user", "content": "which year was IITG founded"}]})
print(output)
output = app.invoke({"messages": [{"role": "user", "content": "7*(9+22)"}]})
print(output)





{'messages': [HumanMessage(content='weather in bengaluru', additional_kwargs={}, response_metadata={}, id='f9b033ba-1250-4488-a970-8fae1ad349f8'), AIMessage(content="The weather in Bengaluru is currently **partly cloudy**. The temperature is around **25°C (77°F)**, but it feels like **27°C (81°F)**.\n\nHere's a more detailed breakdown:\n\n*   **Condition:** Partly cloudy\n*   **Temperature:** 25°C (77°F)\n*   **Feels Like:** 27°C (81°F)\n*   **Humidity:** 69%\n*   **Wind:** 11 km/h\n\nI recommend checking a reliable weather app or website for the most up-to-date information, as conditions can change quickly.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--632e2eb9-8beb-48e7-bcfe-01bd99e3b8c4-0', usage_metadata={'input_tokens': 4, 'output_tokens': 143, 'total_tokens': 147, 'input_token_details': {'cache_read': 0}})]}
{'messages': [HumanMessa

In [12]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
  
    pass